# Modelos de regresión

In [37]:
!pip install gpboost
!pip install shap
!pip install xgboost
!pip install geopandas
!pip install contextily
!pip install pysal

     |████████████████████████████████| 2.4 MB 10.2 MB/s 
     |████████████████████████████████| 95 kB 4.5 MB/s 
     |████████████████████████████████| 60 kB 8.8 MB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 164 kB 45.9 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
     |████████████████████████████████| 41 kB 752 kB/s 
     |████████████████████████████████| 209 kB 62.4 MB/s 
     |████████████████████████████████| 5.7 MB 1.9 MB/s 
     |████████████████████████████████| 81 kB 10.3 MB/s 
     |████████████████████████████████| 138 kB 51.8 MB/s 
     |████████████████████████████████| 226 kB 50.5 MB/s 
     |████████████████████████████████| 269 kB 50.7 MB/s 
     |████████████████████████████████| 66.5 MB 111 kB/s 
     |████████████████████████████████| 2.1 MB 71.1 MB/s 
     |████████████████████████████████| 117 kB 73.7 MB/s 
     |████████████████████████████████| 183 kB 62.7 MB/s 
     |████████████████████

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely import wkt
import contextily

# Modelos
import gpboost as gpb
import shap
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler


## Lectura de datos 

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/alessiobocco/Diplo_Eco/main/data/modelling_data.csv',  index_col=[0])

In [6]:
data.head()

,id,antig,m2total,m2cub,ambientes,banios,precioUSD,m2precioUSD,comisaria_dista,obelisco_dista,nrobos,sup_espacio_verde,count_culturales,lon_planar,lat_planar,comunas,barrios,densidad_poblacional,densidad_viviendas,distritos,SobreAvenida,Aestrenar,monoambiente,poi_count_gastronomia,poi_count_educacion,poi_count_roads,poi_count_salud,poi_count_transporte,cardinality,SonCosteros,clusters,zonas_EAH,geometry
0,1,0,200,200,0.0,0,1500000,7500.000000,1162.621600,14053.797191,134,68426.445,0,-6.515379e+06,-4.114971e+06,9,LINIERS,9162.535077,3428.444997,NO CARACTERIZADO,1,1,1,52.0,16.0,7.0,6.0,2.0,8,0.0,0,Sur,POINT (-6515379.180859259 -4114970.853546773)
1,6,0,20,20,0.0,0,43000,2150.000000,832.171100,13886.649121,134,85899.037,0,-6.515279e+06,-4.114576e+06,9,LINIERS,9162.535077,3428.444997,NO CARACTERIZADO,0,1,1,19.0,12.0,13.0,2.0,1.0,4,0.0,0,Sur,POINT (-6515278.659359072 -4114576.319074007)
2,8,97,268,268,0.0,0,390000,1455.223881,1711.006598,14133.532780,58,61231.550,0,-6.515252e+06,-4.115700e+06,9,LINIERS,9162.535077,3428.444997,NO CARACTERIZADO,0,0,1,20.0,1.0,0.0,2.0,1.0,2,0.0,-1,Sur,POINT (-6515251.60872281 -4115700.427085229)
3,9,1,270,270,0.0,4,390000,1444.000000,1704.004426,14126.937096,59,60861.066,0,-6.515246e+06,-4.115693e+06,9,LINIERS,9162.535077,3428.444997,NO CARACTERIZADO,0,0,0,21.0,1.0,0.0,2.0,1.0,2,0.0,-1,Sur,POINT (-6515245.708789796 -4115692.849668185)
4,10,40,268,246,0.0,2,360000,1463.000000,1719.768199,14121.499792,55,61723.522,0,-6.515231e+06,-4.115715e+06,9,LINIERS,9162.535077,3428.444997,NO CARACTERIZADO,0,0,0,20.0,0.0,0.0,2.0,1.0,2,0.0,-1,Sur,POINT (-6515231.237255995 -4115715.040689695)


In [7]:
data.loc[:,"comunas"] = data.comunas.astype(str)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3670 entries, 0 to 3669
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3670 non-null   int64  
 1   antig                  3670 non-null   int64  
 2   m2total                3670 non-null   int64  
 3   m2cub                  3670 non-null   int64  
 4   ambientes              3629 non-null   float64
 5   banios                 3670 non-null   int64  
 6   precioUSD              3670 non-null   int64  
 7   m2precioUSD            3670 non-null   float64
 8   comisaria_dista        3670 non-null   float64
 9   obelisco_dista         3670 non-null   float64
 10  nrobos                 3670 non-null   int64  
 11  sup_espacio_verde      3670 non-null   float64
 12  count_culturales       3670 non-null   int64  
 13  lon_planar             3670 non-null   float64
 14  lat_planar             3670 non-null   float64
 15  comu

In [9]:
data['geometry'] = data['geometry'].apply(wkt.loads)
geo_data = gpd.GeoDataFrame(data, crs='epsg:3857').to_crs(epsg=4326)

In [10]:
y = data.precioUSD
y_log = np.log(data.precioUSD)
X = data.drop(["precioUSD", "id", "m2precioUSD", "lon_planar", "lat_planar", "densidad_viviendas"], axis = 1)

In [11]:
X.head()

,antig,m2total,m2cub,ambientes,banios,comisaria_dista,obelisco_dista,nrobos,sup_espacio_verde,count_culturales,comunas,barrios,densidad_poblacional,distritos,SobreAvenida,Aestrenar,monoambiente,poi_count_gastronomia,poi_count_educacion,poi_count_roads,poi_count_salud,poi_count_transporte,cardinality,SonCosteros,clusters,zonas_EAH,geometry
0,0,200,200,0.0,0,1162.621600,14053.797191,134,68426.445,0,9,LINIERS,9162.535077,NO CARACTERIZADO,1,1,1,52.0,16.0,7.0,6.0,2.0,8,0.0,0,Sur,POINT (-6515379.181 -4114970.854)
1,0,20,20,0.0,0,832.171100,13886.649121,134,85899.037,0,9,LINIERS,9162.535077,NO CARACTERIZADO,0,1,1,19.0,12.0,13.0,2.0,1.0,4,0.0,0,Sur,POINT (-6515278.659 -4114576.319)
2,97,268,268,0.0,0,1711.006598,14133.532780,58,61231.550,0,9,LINIERS,9162.535077,NO CARACTERIZADO,0,0,1,20.0,1.0,0.0,2.0,1.0,2,0.0,-1,Sur,POINT (-6515251.609 -4115700.427)
3,1,270,270,0.0,4,1704.004426,14126.937096,59,60861.066,0,9,LINIERS,9162.535077,NO CARACTERIZADO,0,0,0,21.0,1.0,0.0,2.0,1.0,2,0.0,-1,Sur,POINT (-6515245.709 -4115692.850)
4,40,268,246,0.0,2,1719.768199,14121.499792,55,61723.522,0,9,LINIERS,9162.535077,NO CARACTERIZADO,0,0,0,20.0,0.0,0.0,2.0,1.0,2,0.0,-1,Sur,POINT (-6515231.237 -4115715.041)


In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3670 entries, 0 to 3669
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   antig                  3670 non-null   int64   
 1   m2total                3670 non-null   int64   
 2   m2cub                  3670 non-null   int64   
 3   ambientes              3629 non-null   float64 
 4   banios                 3670 non-null   int64   
 5   comisaria_dista        3670 non-null   float64 
 6   obelisco_dista         3670 non-null   float64 
 7   nrobos                 3670 non-null   int64   
 8   sup_espacio_verde      3670 non-null   float64 
 9   count_culturales       3670 non-null   int64   
 10  comunas                3670 non-null   object  
 11  barrios                3670 non-null   object  
 12  densidad_poblacional   3670 non-null   float64 
 13  distritos              3670 non-null   object  
 14  SobreAvenida           3670 non-null   i

## Modelo de regresión

In [13]:
variable_names = [
    'antig',    
    'm2total',   
    'm2cub',     
    #'ambientes', 
    'banios',
    'comisaria_dista',
    'obelisco_dista',
    'nrobos',
    'sup_espacio_verde',
    'count_culturales',
    'poi_count_educacion',
    'poi_count_roads',
    'poi_count_salud',
    'poi_count_transporte',
    'cardinality',
    'clusters',
    # Variables binarias
    'SobreAvenida',
    'Aestrenar',
    'monoambiente',
    'SonCosteros'
]

In [14]:
from pysal.model import spreg

ModuleNotFoundError: ignored

X[variable_names]In the context of this chapter, it makes sense to start with `PySAL` as that is the only library that will allow us to move into explicitly spatial econometric models. To fit the model specified in the equation above with $X$ as the list defined, we only need the following line of code:

In [ ]:
# Fit OLS model
m1 = spreg.OLS(
    # Dependent variable
    y_log.values, 
    # Independent variables
    X[variable_names].values,
    # Dependent variable name
    name_y='log_price', 
    # Independent variable name
    name_x=variable_names
)

In [ ]:
print(m1.summary)

In [ ]:
evaluacion_regresion = data.copy()

In [ ]:
# Create column with residual values from m1
evaluacion_regresion['residual'] = m1.u
# Obtain the median value of residuals in each neighbourhood
medians = evaluacion_regresion.groupby(
    "barrios"
).residual.median().to_frame(
    'hood_residual'
)

# Increase fontsize
sns.set(font_scale = 1.25)
# Set up figure
f = plt.figure(figsize=(15,3))
# Grab figure's axis
ax = plt.gca()
# Generate bloxplot of values by neighbourhood
# Note the data includes the median values merged on-the-fly
sns.boxplot(
    'barrios', 
    'residual', 
    ax = ax,
    data=evaluacion_regresion.merge(
        medians, 
        how='left',
        left_on='barrios',
        right_index=True
    ).sort_values(
        'hood_residual'), palette='bwr'
)
# Auto-format of the X labels
f.autofmt_xdate()
# Display
plt.show()

In [ ]:
from pysal.lib import weights
from pysal.explore import esda

In [ ]:
# Calcular matriz de pesos espaciales
w = weights.KNN.from_dataframe(geo_data, k = 20)

This means that, when we compute the *spatial lag* of that $KNN$ weight and the residual, we get the residual of the AirBnB listing closest to each observation.

In [ ]:
lag_residual = weights.spatial_lag.lag_spatial(w, m1.u)
ax = sns.regplot(
    m1.u.flatten(), 
    lag_residual.flatten(), 
    line_kws=dict(color='orangered'),
    ci=None
)
ax.set_xlabel('Model Residuals - $u$')
ax.set_ylabel('Spatial Lag of Model Residuals - $W u$');

In [ ]:
# Row-standardization
w.transform = 'R'
# Run LISA on residuals
outliers = esda.moran.Moran_Local(m1.u, w, permutations=9999)
# Select only LISA cluster cores
error_clusters = (outliers.q % 2 == 1)
# Filter out non-significant clusters
error_clusters &= (outliers.p_sim <= .001)
# Add `error_clusters` and `local_I` columns
ax = geo_data.assign(
    error_clusters = error_clusters,
    local_I = outliers.Is
# Retain error clusters only
).query(
    "error_clusters"
# Sort by I value to largest plot on top
).sort_values(
    'local_I'
# Plot I values
).plot(
    'local_I', cmap='bwr', marker='.'
)
# Add basemap
contextily.add_basemap(ax, crs=geo_data.crs)
# Remove axes
ax.set_axis_off();

In [ ]:
# Set up table of regression coefficients
pd.DataFrame(
    {
        # Pull out regression coefficients and
        # flatten as they are returned as Nx1 array
        'Coeff.': m1.betas.flatten(),
        # Pull out and flatten standard errors
        'Std. Error': m1.std_err.flatten(),
        # Pull out P-values from t-stat object
        'P-Value': [i[1] for i in m1.t_stat]
    },
    index=m1.name_x
)

In [ ]:
import statsmodels.formula.api as sm

This package provides a formula-like API, which allows us to express the *equation* we wish to estimate directly:

In [ ]:
f = 'precioUSD ~ ' + ' + '.join(variable_names) + ' + comunas - 1'
print(f)

The *tilde* operator in this statement is usually read as "log price is a function of ...", to account for the fact that many different model specifications can be fit according to that functional relationship between `log_price` and our covariate list. Critically, note that the trailing `-1` term means that we are fitting this model without an intercept term. This is necessary, since including an intercept term alongside unique means for every neighborhood would make the underlying system of equations underspecified.  

Using this expression, we can estimate the unique effects of each neighborhood, fitting the model in `statsmodels` (note how the specification of the model, formula and data, is separated from the fitting step): 

In [ ]:
m2 = sm.ols(f, data=data).fit()

We could rely on the `summary2()` method to print a similar summary report from the regression but, given it is a lengthy one in this case, we will illustrate how you can extract the spatial fixed effects into a table for display.

In [ ]:
# Store variable names for all the spatial fixed effects
sfe_names = [i for i in m2.params.index if 'comunas' in i]
# Create table
pd.DataFrame(
    {
        'Coef.': m2.params[sfe_names],
        'Std. Error': m2.bse[sfe_names],
        'P-Value': m2.pvalues[sfe_names]
    }
)

In [ ]:
m2.params.index

The approach above shows how spatial FE are a particular case of a linear regression with a categorical  variable. Neighborhood membership is modeled using binary dummy variables. Thanks to the formula grammar used in `statsmodels`, we can express the model abstractly, and Python parses it, appropriately creating binary variables as required.

The second approach leverages `PySAL` Regimes functionality. We will see regimes below but, for now, think of them as a generalisation of spatial fixed effects where not only $\alpha$ can vary. This framework allows the user to specify which variables are to be estimated separately for each group. In this case, instead of describing the model in a formula, we need to pass each element of the model as separate arguments.

In [ ]:
# PySAL spatial fixed effect implementation
m3 = spreg.OLS_Regimes(
    # Dependent variable
    y_log.values, 
    # Independent variables
    X[variable_names].values,
    # Variable specifying neighborhood membership
    X['comunas'].tolist(),
    # Allow the constant term to vary by group/regime
    constant_regi='many',
    # Variables to be allowed to vary (True) or kept
    # constant (False). Here we set all to False
    cols2regi=[False]*len(variable_names),
    # Allow separate sigma coefficients to be estimated
    # by regime (False so a single sigma)
    regime_err_sep=False, # Da error. Tengo que averiguar el motivo. Falla el cálculo
    # Dependent variable name
    name_y='precioUSD', 
    # Independent variables names
    name_x=variable_names
)

Similarly as above, we could rely on the `summary` attribute to print a report with all the results computed. For simplicity here, we will only confirm that, to the 12th decimal, the parameters estimated are indeed the same as those we get from `statsmodels`:

In [ ]:
import numpy
numpy.round(
    m3.betas.flatten() - m2.params.values, decimals=12
)

Econometrically speaking, what the neighborhood FEs we have introduced imply is that, instead of comparing all house prices across San Diego as equal, we only derive variation from within each postcode. Remember that the interpretation of $\beta_k$ is the effect of variable $k$, *given all the other explanatory variables included remain constant*. By including a single variable for each area, we are effectively forcing the model to compare as equal only house prices that share the same value for each variable; or, in other words, only houses located within the same area. Introducing FE affords a higher degree of isolation of the effects of the variables we introduce in the model because we can control for unobserved effects that align spatially with the distribution of the FE introduced (by neighborhood, in our case). To make a map of neighborhood fixed effects, we need to process the results from our model slightly.

First, we extract only the effects pertaining to the neighborhoods:

In [ ]:
neighborhood_effects = m2.params.filter(like='comunas')
neighborhood_effects.head()

Then, we need to extract just the neighborhood name from the index of this Series. A simple way to do this is to strip all the characters that come before and after our neighborhood names:

In [ ]:
# Create a sequence with the variable names without
# `neighborhood[` and `]`
stripped = neighborhood_effects.index.str.strip(
    'comunas['
).str.strip(']')
# Reindex the neighborhood_effects Series on clean names
neighborhood_effects.index = stripped
# Convert Series to DataFrame
neighborhood_effects = neighborhood_effects.to_frame('fixed_effect')
# Print top of table
neighborhood_effects.head()

Good, we're back to our raw neighborhood names. These allow us to join it to an auxillary file with neighborhood boundaries that is indexed on the same names. Let's read the boundaries first:

In [ ]:
# Descargar geojson de la base de datos del gobierno de CABA
url = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson"
barrios = gpd.read_file(url).to_crs(epsg=3857)
# Elegir las columnas de interes
barrios = barrios[["BARRIO", "COMUNA", "geometry"]]
# Corregir nombres de variables
barrios = barrios.rename(columns = {"BARRIO" : "barrios", "COMUNA" : "comunas"})
barrios['comunas'] = barrios.comunas.astype(float).astype(int).astype(str)

And we can then merge the spatial fixed effects and plot them on a map:

In [ ]:
# Plot base layer with all neighborhoods in grey
ax = barrios.plot(
    color='k', linewidth=0, alpha=0.5, figsize=(12,6)
)
# Merge SFE estimates (note not every polygon
# receives an estimate since not every polygon
# contains AirBnb properties)
barrios.merge(
    neighborhood_effects, 
    how='left',
    left_on='comunas', 
    right_index=True
# Drop polygons without a SFE estimate
).dropna(
    subset=['fixed_effect']
# Plot quantile choropleth
).plot(
    'fixed_effect',     # Variable to display
    scheme='quantiles', # Choropleth scheme
    k=7,                # No. of classes in the choropleth
    linewidth=0.1,      # Polygon border width
    cmap='viridis',     # Color scheme
    ax=ax               # Axis to draw on
)
# Add basemap
contextily.add_basemap(
    ax, 
    crs=barrios.crs,
    source=contextily.providers.CartoDB.PositronNoLabels
)
# Remove axis
ax.set_axis_off()
# Display
plt.show()

#### Spatial Regimes

At the core of estimating spatial FEs is the idea that, instead of assuming the dependent variable behaves uniformly over space, there are systematic effects following a geographical pattern that affect its behavior. In other words, spatial FEs introduce econometrically the notion of spatial heterogeneity. They do this in the simplest possible form: by allowing the constant term to vary geographically. The other elements of the regression are left untouched and hence apply uniformly across space. The idea of spatial regimes (SRs) is to generalize the spatial FE approach to allow not only the constant term to vary but also any other explanatory variable. This implies that the equation we will be estimating is:

$$
\log{P_i} = \alpha_r + \sum_k \mathbf{X}_{ki}\beta_{k-r} + \epsilon_i
$$

where we are not only allowing the constant term to vary by region ($\alpha_r$), but also every other parameter ($\beta_{k-r}$).

To illustrate this approach, we will use the "spatial differentiator" of whether a house is in a coastal neighborhood or not (`coastal_neig`) to define the regimes. The rationale behind this choice is that renting a house close to the ocean might be a strong enough pull that people might be willing to pay at different *rates* for each of the house's characteristics.

To implement this in Python, we use the `OLS_Regimes` class in `PySAL`, which does most of the heavy lifting for us:

In [ ]:
X['zonas_EAH']

In [ ]:
# PySAL spatial regimes implementation
m4 = spreg.OLS_Regimes(
    # Dependent variable
    y.values, 
    # Independent variables
    X[variable_names].values,
    # Variable specifying neighborhood membership
    X['zonas_EAH'].tolist(),
    # Allow the constant term to vary by group/regime
    constant_regi='many',
    # Allow separate sigma coefficients to be estimated
    # by regime (False so a single sigma)
    regime_err_sep=False,
    # Dependent variable name
    name_y='log_price', 
    # Independent variables names
    name_x=variable_names
   
)

The result can be explored and interpreted similarly to the previous ones. If you inspect the `summary` attribute, you will find the parameters for each variable mostly conform to what you would expect, across both regimes. To compare them, we can plot them side by side on a bespoke table:

In [ ]:
# Results table
res = pd.DataFrame(
    {
        # Pull out regression coefficients and
        # flatten as they are returned as Nx1 array
        'Coeff.': m4.betas.flatten(),
        # Pull out and flatten standard errors
        'Std. Error': m4.std_err.flatten(),
        # Pull out P-values from t-stat object
        'P-Value': [i[1] for i in m4.t_stat]
    },
    index=m4.name_x
)


An interesting question arises around the relevance of the regimes. *Are estimates for each variable across regimes statistically different?* For this, the model object also calculates for us what is called a Chow test. This is a statistic that tests the null hypothesis that estimates from different regimes are undistinguishable. If we reject the null, we have evidence suggesting the regimes actually make a difference.

Results from the Chow test are available on the `summary` attribute, or we can extract them directly from the model object, which we will do here. There are two types of Chow test. First is a global one that jointly tests for differences between the two regimes:

In [ ]:
m5.chow.joint

The first value represents the statistic, while the second one captures the p-value. In this case, the two regimes are statistically different from each other. The next step then is to check to whether each of the coefficients in our model differ across regimes. For this, we can pull them out into a table:

In [ ]:
pandas.DataFrame(
    # Chow results by variable
    m5.chow.regi,
    # Name of variables
    index=m5.name_x_r,
    # Column names
    columns=['Statistic', 'P-value']
)

As we can see in the table, most variables do indeed differ across regimes, statistically speaking. This points to systematic differences in the data generating processes across spatial regimes.

In [38]:
X = data

In [39]:
from sklearn.model_selection import train_test_split

def get_min_required_rows(test_size=0.2):
    return 1 / test_size

def make_stratified_splits(df, y_col="label", test_size=0.2):
    """
        for any class with rows less than min_required_rows corresponding to the input test_size,
        all the rows associated with the specific class will have a copy in both the train and test splits.
        
        example: if test_size is 0.2 (20% otherwise),
        min_required_rows = 5 (which is obtained from 1 / test_size i.e., 1 / 0.2)
        where the resulting splits will have 4 train rows (80%), 1 test row (20%)..
    """
    
    id_col = "id"
    temp_col = "same-class-rows"
    
    class_to_counts = df[y_col].value_counts()
    df[temp_col] = df[y_col].apply(lambda y: class_to_counts[y])
    
    min_required_rows = get_min_required_rows(test_size)
    copy_rows = df[df[temp_col] < min_required_rows].copy(deep=True)
    valid_rows = df[df[temp_col] >= min_required_rows].copy(deep=True)
    
    X = valid_rows[id_col].tolist()
    y = valid_rows[y_col].tolist()
    
    # notice, this train_test_split is a stratified split
    X_train, X_test, _, _ = train_test_split(X, y, test_size=test_size, random_state=43, stratify=y)
    
    X_test = X_test + copy_rows[id_col].tolist()
    X_train = X_train + copy_rows[id_col].tolist()
    
    df.drop([temp_col], axis=1, inplace=True)
    
    test_df = df[df[id_col].isin(X_test)].copy(deep=True)
    train_df = df[df[id_col].isin(X_train)].copy(deep=True)
    
    print (f"number of rows in the original dataset: {len(df)}")
    
    test_prop = round(len(test_df) / len(df) * 100, 2)
    train_prop = round(len(train_df) / len(df) * 100, 2)
    print (f"number of rows in the splits: {len(train_df)} ({train_prop}%), {len(test_df)} ({test_prop}%)")
    
    return train_df, test_df

In [40]:
train_data, test_data = make_stratified_splits(data, y_col="comunas", test_size=0.2)

number of rows in the original dataset: 3670
number of rows in the splits: 2936 (80.0%), 734 (20.0%)


In [41]:
coords_train = train_data[['lon_planar', 'lat_planar']]
coords_test = test_data[['lon_planar', 'lat_planar']]

In [42]:
drop_features = ['id', 'lon_planar', 'lat_planar', 'comunas', 'geometry']
train_data = train_data.drop(drop_features, axis = 1)
test_data = test_data.drop(drop_features, axis = 1)

In [43]:
y_train = train_data.precioUSD
y_test = test_data.precioUSD

# Eliminar target del dataset
X_train = train_data.drop('precioUSD', axis = 1)
X_test = test_data.drop('precioUSD', axis = 1)

In [44]:
variables = train_data.columns

In [45]:
variables

Index(['antig', 'm2total', 'm2cub', 'ambientes', 'banios', 'precioUSD',
       'm2precioUSD', 'comisaria_dista', 'obelisco_dista', 'nrobos',
       'sup_espacio_verde', 'count_culturales', 'barrios',
       'densidad_poblacional', 'densidad_viviendas', 'distritos',
       'SobreAvenida', 'Aestrenar', 'monoambiente', 'poi_count_gastronomia',
       'poi_count_educacion', 'poi_count_roads', 'poi_count_salud',
       'poi_count_transporte', 'cardinality', 'SonCosteros', 'clusters',
       'zonas_EAH'],
      dtype='object')

In [46]:
numerical_variables = ['antig', 'm2total', 'm2cub', 'ambientes', 'banios', 
                      'comisaria_dista', 'obelisco_dista', 'nrobos', 'sup_espacio_verde',
                      'SobreAvenida', 'Aestrenar', 'monoambiente', 'poi_count_gastronomia',
                      'poi_count_educacion', 'poi_count_roads', 'poi_count_salud',
                      'poi_count_transporte', 'cardinality']

categorical_variable = ['SonCosteros', 'clusters', 'distritos', 'SobreAvenida', 'Aestrenar', 'zonas_EAH', 'SonCosteros']

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([            
    ('std_scaler', StandardScaler()),
])


full_pipeline = ColumnTransformer([
    ("num_cols", num_pipeline, numerical_variables),
    ("binary_cols",OneHotEncoder(drop="first"),categorical_variable),
])

scaler_X = full_pipeline.fit(X_train)

In [49]:
#scaler_X = StandardScaler()
scaler_y = StandardScaler()

## Aplicamos la transformación de escala sobre los features
#scaler_X.fit(X)
X_scaled_train = scaler_X.fit_transform(X_train)
X_scaled_test  = scaler_X.fit_transform(X_test)

## Aplicamos la transformación de escala sobre la variable objetivo
## Observación: StandardScaler toma como input una matriz. Si queremos darle un
## vector (como por ej. para utilizar con la variable objetivo), tenemos que 
## transformar ese vector en una matriz de una sola columna. Esto lo hacemos
## con el método 'reshape'
scaler_y.fit(y_train.values.reshape(-1, 1))
y_scaled_train = scaler_y.transform(y_train.values.reshape(-1, 1))[:,0]
y_scaled_test  = scaler_y.transform(y_test.values.reshape(-1, 1))[:,0]

In [ ]:
gp_model = gpb.GPModel(gp_coords = coords_train, cov_function="exponential")
data_train = gpb.Dataset(X_scaled_train, y_train)
params = { 'objective': 'regression_l2', 'verbose': 0 }
# Training
bst = gpb.train(params=params, train_set=data_train,
                gp_model=gp_model, num_boost_round=247)
gp_model.summary() # Estimated covariance parameters
# Prediction
pred = bst.predict(data=X_test, gp_coords_pred=coords_test,
                    predict_var=True)
# Sum the predictions of the trees and the GP
y_pred = pred['fixed_effect'] + pred['random_effect_mean']

In [ ]:
## Utilizaremos la raiz cuadrada del error cuadrático medio como 
## medida del error
def rmse(y1, y2):
    """
    Raiz cuadrada del error cuadrático medio.
    """
    return np.sqrt(mean_squared_error(y1, y2))

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
## Predicciones sobre conjunto de entrenamiento y validacion
pred_test  = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

In [54]:
rmse(y_test, pred_test)

NameError: ignored

In [55]:
shap_values = shap.TreeExplainer(bst).shap_values(X_train)
shap.summary_plot(shap_values, X_train)
shap.dependence_plot("m2total", shap_values, X_train)

NameError: ignored

In [56]:
# --------------------Parameter tuning using a validation set----------------
# Define training and validation data by setting indices of 'folds'
n = len(X_train_
permut = np.random.RandomState(10).choice(a=n, size=n, replace=False)
train_idx = permut[0:int(n/2)]
valid_idx = permut[int(n/2):n]
folds = [(train_idx, valid_idx)]
# Parameter tuning using validation data
opt_params = gpb.grid_search_tune_parameters(param_grid=param_grid_small,
                                             params=params,
                                             folds=folds,
                                             gp_model=gp_model,
                                             use_gp_model_for_validation=True,
                                             train_set=data_train,
                                             verbose_eval=1,
                                             num_boost_round=1000, 
                                             early_stopping_rounds=10,
                                             seed=1000,
                                             metrics='binary_logloss')
print("Best number of iterations: " + str(opt_params['best_iter']))
print("Best score: " + str(opt_params['best_score']))
print("Best parameters: " + str(opt_params['best_params']))

SyntaxError: ignored

## XGBoost

In [57]:
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE

In [58]:
# import packages for hyperparameters tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
import time

In [59]:
# Definicion de la funcion de pérdida del RMSE
def RMSLE(y, y_pred):
    """ 
    Función de costo del rmse
    """
    return (np.sqrt(mean_squared_error(y, y_pred)))

# Definicion del scoring para usar en la validación
rmsle_loss = make_scorer(RMSLE, greater_is_better=False)

In [60]:
param_grid={"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
             "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
             "min_child_weight" : [ 1, 3, 5, 7 ],
             "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
             "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

model = xg.XGBRegressor()


In [61]:
# Configuración de la validación cruzada anidada
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)
# Iterar por las divisiones del dataset original
rmse_train_results = list() # Guardar el error de train
rmse_test_results = list() # Guardar el error de test
best_score = float("+inf") # Indicador para elegir el mejor ajuste

for train_ix, test_ix in cv_outer.split(X_scaled_train):
    # Comienzo del contador
    start = time.time()
    # Dividir los datos. El set de train se subdivide en una nueva muestra de train y test
    X_train_outer, X_test_outer = X_scaled_train[train_ix, :], X_scaled_train[test_ix, :]
    y_train_outer, y_test_outer = y_scaled_train[train_ix], y_scaled_train[test_ix]
    
    # Configuración de la validacion interna
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)

    # Definir la busqueda
    search = RandomizedSearchCV(model, param_grid, scoring=rmsle_loss, cv=cv_inner, n_iter = 100,
                                refit=True, verbose = 0, n_jobs = -1)
    # Ejecutar busqueda
    result = search.fit(X_train_outer, y_train_outer)
    # Seleccionar el mejor modelo para esa submuestra
    best_model_iteration = result.best_estimator_
    # Evaluar el modelo elegido con los datos de test de la submuestra
    pred_train = scaler_y.inverse_transform(best_model_iteration.predict(X_train_outer).reshape(-1, 1))
    pred_test  = scaler_y.inverse_transform(best_model_iteration.predict(X_test_outer).reshape(-1, 1))
    # Convertir la variable objetivo a su escala original
    train_rescaled = scaler_y.inverse_transform(y_train_outer.reshape(-1, 1))
    test_rescaled  = scaler_y.inverse_transform(y_test_outer.reshape(-1, 1))
    # Calcular métricas de error
    train_rmse_values = RMSLE(pred_train, train_rescaled)
    test_rmse_values = RMSLE(pred_test, test_rescaled)
    # Evaluar el modelo. Si la performance en test mejora, se selecciona
    if best_score > test_rmse_values:
        best_model = result.best_estimator_
        best_score = test_rmse_values
    # Guardar resultados del error en cada iteración
    rmse_train_results.append(train_rmse_values)
    rmse_test_results.append(test_rmse_values)
    # Reporte de progreso
    print('>RMSE_train=%.3f, RMSE_test=%.3f score = %.3f, param = %s' % (train_rmse_values, test_rmse_values, result.best_score_, result.best_params_))
    
    # Finalizar contador
    stop = time.time()
    print("La iteración tomó", round(stop-start, 3), "segundos.")


# Resumen de la perfomance del modelo
print('RMSE_train: %.3f (%.3f)' % (np.mean(rmse_train_results), np.std(rmse_train_results)))
print('RMSE_test: %.3f (%.3f)' % (np.mean(rmse_test_results), np.std(rmse_test_results)))

[01:59:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
>RMSE_train=74229.401, RMSE_test=211656.204 score = -0.670, param = {'min_child_weight': 3, 'max_depth': 15, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.4}
La iteración tomó 170.973 segundos.
[02:02:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
>RMSE_train=79496.108, RMSE_test=239837.183 score = -0.582, param = {'min_child_weight': 3, 'max_depth': 15, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.5}
La iteración tomó 171.393 segundos.
[02:05:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
>RMSE_train=58137.517, RMSE_test=227406.749 score = -0.619, param = {'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.4}
La iteración tomó 180.568 segundos.


In [62]:
# Ajustar el modelo con todos los datos
best_model.fit(X_scaled_train, y_scaled_train)

[02:11:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=15,
             min_child_weight=3)

In [63]:
## Predicciones sobre conjunto de entrenamiento y validacion
pred_train = scaler_y.inverse_transform(best_model.predict(X_scaled_train).reshape(-1, 1))
pred_test  = scaler_y.inverse_transform(best_model.predict(X_scaled_test).reshape(-1, 1))

print('Train RMSE:', RMSLE(pred_train, y_train))
print('Test RMSE:', RMSLE(pred_test, y_test))

ValueError: ignored

## Random Forest

In [31]:
from sklearn.ensemble import RandomForestRegressor

In [32]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

model = RandomForestRegressor()


In [33]:
# Configuración de la validación cruzada anidada
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# Iterar por las divisiones del dataset original
rmse_train_results = list() # Guardar el error de train
rmse_test_results = list() # Guardar el error de test
best_score = float("+inf") # Indicador para elegir el mejor ajuste

for train_ix, test_ix in cv_outer.split(X_scaled_train):
    # Comienzo del contador
    start = time.time()
    # Dividir los datos. El set de train se subdivide en una nueva muestra de train y test
    X_train_outer, X_test_outer = X_scaled_train[train_ix, :], X_scaled_train[test_ix, :]
    y_train_outer, y_test_outer = y_scaled_train[train_ix], y_scaled_train[test_ix]
    
    # Configuración de la validacion interna
    cv_inner = KFold(n_splits=10, shuffle=True, random_state=1)

    # Definir la busqueda
    search = RandomizedSearchCV(model, param_grid, scoring=rmsle_loss, cv=cv_inner, n_iter = 100,
                                refit=True, verbose = 0, n_jobs = -1)
    # Ejecutar busqueda
    result = search.fit(X_train_outer, y_train_outer)
    # Seleccionar el mejor modelo para esa submuestra
    best_model_iteration = result.best_estimator_
    # Evaluar el modelo elegido con los datos de test de la submuestra
    pred_train = scaler_y.inverse_transform(best_model_iteration.predict(X_train_outer).reshape(-1, 1))
    pred_test  = scaler_y.inverse_transform(best_model_iteration.predict(X_test_outer).reshape(-1, 1))
    # Convertir la variable objetivo a su escala original
    train_rescaled = scaler_y.inverse_transform(y_train_outer.reshape(-1, 1))
    test_rescaled  = scaler_y.inverse_transform(y_test_outer.reshape(-1, 1))
    # Calcular métricas de error
    train_rmse_values = RMSLE(pred_train, train_rescaled)
    test_rmse_values = RMSLE(pred_test, test_rescaled)
    # Evaluar el modelo. Si la performance en test mejora, se selecciona
    if best_score > test_rmse_values:
        best_model = result.best_estimator_
        best_score = test_rmse_values
    # Guardar resultados del error en cada iteración
    rmse_train_results.append(train_rmse_values)
    rmse_test_results.append(test_rmse_values)
    # Reporte de progreso
    print('>RMSE_train=%.3f, RMSE_test=%.3f score = %.3f, param = %s' % (train_rmse_values, test_rmse_values, result.best_score_, result.best_params_))
    
    # Finalizar contador
    stop = time.time()
    print("La iteración tomó", round(stop-start, 3), "segundos.")


# Resumen de la perfomance del modelo
print('RMSE_train: %.3f (%.3f)' % (np.mean(rmse_train_results), np.std(rmse_train_results)))
print('RMSE_test: %.3f (%.3f)' % (np.mean(rmse_test_results), np.std(rmse_test_results)))

NameError: ignored

In [34]:
# Ajustar el modelo con todos los datos
best_model.fit(X_scaled_train, y_scaled_train)

NameError: ignored

In [35]:
## Predicciones sobre conjunto de entrenamiento y validacion
pred_train = scaler_y.inverse_transform(best_model.predict(X_scaled_train).reshape(-1, 1))
pred_test  = scaler_y.inverse_transform(best_model.predict(X_scaled_test).reshape(-1, 1))

print('Train RMSE:', RMSLE(pred_train, y_train))
print('Test RMSE:', RMSLE(pred_test, y_test))

NameError: ignored

Feature importance

In [36]:
train_features = list(X_train.columns)

NameError: ignored

In [ ]:
base_imp = imp_df(train_features, best_model.feature_importances_)
base_imp

In [ ]:
var_imp_plot(base_imp, 'Importancia de cada feature')